In [1]:
import csv
import cv2
import numpy as np

In [2]:
"""Extract data from the .csv file"""
lines = []
with open('./data/driving_log.csv') as file:
    reader = csv.reader(file)
    for line in reader:
        lines.append(line)
# images and mearsurments will be our input_X and input_y
# Augment training set by flip the image horizonaly and flip the sign of mearsurment.
augmented_images = []
augmented_mearsurements = []
for i in range(len(lines)):
    if i == 0:
        continue
    line = lines[i]
    for j in range(3):
        # source path could be center(0)/left(1)/right(2)
        source_path = line[j]
        filename = source_path.split('/')[-1]
        current_path = './data/IMG/' + filename
        image = cv2.imread(current_path)
        augmented_images.append(image)
        augmented_images.append(cv2.flip(image, 1))
        if j == 0:
            mearsurement = float(line[3])
        elif j == 1:
            mearsurement = float(line[3]) + 0.2
        elif j == 2:
            mearsurement = float(line[3]) - 0.2
        augmented_mearsurements.append(mearsurement)
        augmented_mearsurements.append(mearsurement * -1.0)


# for i in range(len(lines)):
#     if i == 0:
#         continue
#     line = lines[i]
#     source_path = line[0]
#     filename = source_path.split('/')[-1]
#     current_path = './data/IMG/' + filename
#     image = cv2.imread(current_path)
#     augmented_images.append(image)
#     augmented_images.append(cv2.flip(image, 1))
#     mearsurement = float(line[3])
#     augmented_mearsurements.append(mearsurement)
#     augmented_mearsurements.append(mearsurement * -1.0)

In [3]:
"""Set X_train and y_train"""
X_train = np.array(augmented_images)
y_train = np.array(augmented_mearsurements)
print(X_train.shape, y_train.shape)

(48216, 160, 320, 3) (48216,)


In [4]:
print(y_train[0:12])

[ 0.  -0.   0.2 -0.2 -0.2  0.2  0.  -0.   0.2 -0.2 -0.2  0.2]


In [11]:
"""Build a simple regression model"""
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Conv2D, MaxPooling2D, Cropping2D
from keras import backend as K
from keras.callbacks import EarlyStopping, TensorBoard  
from keras.preprocessing.image import ImageDataGenerator

model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
# crop 70 rows of pixels on the top and 25 rows of pixels on the bottom.
model.add(Cropping2D(cropping=((70, 25), (0, 0))))
model.add(Conv2D(24, (5, 5), activation="relu", strides=(2, 2)))     # subsample: stride
# model.add(MaxPooling2D())
model.add(Conv2D(36, (5, 5), activation="relu", strides=(2, 2)))
# model.add(MaxPooling2D())
model.add(Conv2D(48, (5, 5), activation="relu", strides=(2, 2)))
# model.add(MaxPooling2D())
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
# output regression layer
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_5 (Lambda)            (None, 160, 320, 3)       0         
_________________________________________________________________
cropping2d_5 (Cropping2D)    (None, 65, 320, 3)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 31, 158, 24)       1824      
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 14, 77, 36)        21636     
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 5, 37, 48)         43248     
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 3, 35, 64)         27712     
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 1, 33, 64)         36928     
__________

In [12]:
datagen = ImageDataGenerator()
datagen.fit(X_train)

callbacks = [EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')]

model.fit_generator(datagen.flow(x_train, y_train, batch_size=128), 
          validation_split=0.2, 
          shuffle=True, 
          epochs=10,
          callbacks=callbacks)



model.save("./models/model_aug_3_2e.h5")



MemoryError: 

In [ ]:
K.clear_session()